In [1]:
from mqt.yaqs.core.data_structures.networks import MPS, MPO
from mqt.yaqs import simulator
from mqt.yaqs.core.data_structures.simulation_parameters import AnalogSimParams, Observable
from mqt.yaqs.core.libraries.gate_library import X, Z, RuntimeCost
from mqt.yaqs.core.data_structures.noise_model import NoiseModel

import matplotlib.pyplot as plt
import numpy as np
import pickle

In [ ]:
def tdvp_simulator(H_0, noise_model, state=None):
    L = H_0.length

    # Ising
    state = MPS(length=L)

    # Heisenberg
    # elapsed_time = 2
    # state = MPS(length=L, state="Neel")

    measurements = [Observable(Z(), [0])] + [Observable("entropy", [L//2, L//2+1])] + [Observable("schmidt_spectrum", [L//2, L//2+1])] + [Observable("max_bond")]
    sim_params = AnalogSimParams(observables=measurements,
                                elapsed_time=5,
                                dt=0.01,
                                num_traj=10,
                                threshold=1e-3,
                                trunc_mode="relative",
                                sample_timesteps=True)

    simulator.run(state, H_0, sim_params, noise_model=noise_model)
    print("Obs Exp Val", sim_params.observables[0].results[-1])
    print("Entropy", sim_params.observables[1].results[-1])
    print("Max Bond", sim_params.observables[3].results[-1])
    return sim_params.observables[1::]

In [ ]:
L_list = [8]
for L in L_list:
    print(L)
    J = 1
    h = 1
    H_0 = MPO()
    H_0.init_ising(L, J, h)
    # H_0.init_heisenberg(L, J, J, J, h)

    gammas = np.logspace(-3, 2, 30)
    results = []
    for j, gamma in enumerate(gammas):
        print(j+1, "of", len(gammas))
        # Define the noise model
        # noise_model = NoiseModel([
        #     {"name": name, "sites": [i], "strength": gamma} for i in range(L) for name in ["pauli_z", "pauli_x", "pauli_y"]
        #     ])
        noise_model = NoiseModel([
            {"name": name, "sites": [i], "strength": gamma} for i in range(L) for name in ["measure_0", "measure_1"]
        ])
        cost = tdvp_simulator(H_0, noise_model)
        results.append(cost)

    filename = f"timestep01_cost_{L}.pickle"
    with open(filename, 'wb') as handle:
        pickle.dump(results, handle)

32
1 of 30


Running trajectories:   0%|                              | 0/10 [00:00<?, ?it/s]